In [10]:
import pandas as pd
import dateparser
import datetime as datetime
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
from textblob import TextBlob
from bs4 import BeautifulSoup

In [4]:
df_yt = pd.read_csv('df_yt.csv')
df_yt_scraped2 = pd.read_csv('df_yt_scraped2.csv')
df_yt_scraped3 = pd.read_csv('df_yt_scraped3.csv')
df_yt_scraped4 = pd.read_csv('df_yt_scraped4.csv')
df_yt_scraped5 = pd.read_csv('df_yt_scraped5.csv')
#df_yt_scraped6 = pd.read_csv('df_yt_scraped6.csv')
#df_yt_scraped7 = pd.read_csv('df_yt_scraped7.csv')
#df_yt_scraped8 = pd.read_csv('df_yt_scraped8.csv')
#df_yt_scraped9 = pd.read_csv('df_yt_scraped9.csv')
df_yt_scraped10 = pd.read_csv('df_yt_scraped10.csv')
#df_yt_scraped11 = pd.read_csv('df_yt_scraped11.csv')
df_yt_scraped12 = pd.read_csv('df_yt_scraped12.csv')


In [32]:
df_comments = pd.concat([df_yt_scraped2, df_yt_scraped3, df_yt_scraped4, df_yt_scraped5, df_yt_scraped10, df_yt_scraped12], ignore_index=True)

In [30]:
df_comments

,Unnamed: 0,Username,Comment,Likes,Replies,Date,link
0,0,@michaelporter-lo9qc,i have a very hard life . im sick and im lost ...,383,100,"<a class=""yt-simple-endpoint style-scope yt-fo...",https://www.youtube.com/watch?v=oz2n-rQCWzI
1,1,@walterf6993,I truly believe people are being saved by your...,76,4,"<a class=""yt-simple-endpoint style-scope yt-fo...",https://www.youtube.com/watch?v=oz2n-rQCWzI
2,2,@gretchen9716,I love your discernment of these times! Your t...,31,1,"<a class=""yt-simple-endpoint style-scope yt-fo...",https://www.youtube.com/watch?v=oz2n-rQCWzI
3,3,@melissalarson2569,A couple years ago I couldn't see how the end ...,44,15,"<a class=""yt-simple-endpoint style-scope yt-fo...",https://www.youtube.com/watch?v=oz2n-rQCWzI
4,4,@joenorthpeigan3197,Praise the Lord. And Thank you pastor John. T...,19,0,"<a class=""yt-simple-endpoint style-scope yt-fo...",https://www.youtube.com/watch?v=oz2n-rQCWzI
...,...,...,...,...,...,...,...
38277,5449,Ever4U,I I'll go to Mall to watch items,0,0,"<a class=""yt-simple-endpoint style-scope yt-fo...",https://www.youtube.com/watch?v=6490tKrGEic
38278,5450,Muneer Hasan,Do lpu really have such kind of teachers...?,0,1,"<a class=""yt-simple-endpoint style-scope yt-fo...",https://www.youtube.com/watch?v=6490tKrGEic
38279,5451,Soumyaranjan Mendili,Hii sri,0,0,"<a class=""yt-simple-endpoint style-scope yt-fo...",https://www.youtube.com/watch?v=6490tKrGEic
38280,5452,Yuta Itadori,Bakbakbak,0,0,"<a class=""yt-simple-endpoint style-scope yt-fo...",https://www.youtube.com/watch?v=6490tKrGEic


In [ ]:
df_comments = pd.concat([df_yt_scraped2, df_yt_scraped3, df_yt_scraped4, df_yt_scraped5, df_yt_scraped6, df_yt_scraped7, df_yt_scraped8, df_yt_scraped9, df_yt_scraped10], ignore_index=True)

In [33]:
def cleaning_youtube_videos(df):
    # Initialize SentimentIntensityAnalyzer
    sia = SentimentIntensityAnalyzer()

    try:
        df = df[df['date'] != 'Unknown'].reset_index(drop=True)
    except:
        pass

    if df['date'].notnull().all():
        df['date'] = df['date'].astype(str)

    # apply the dateparser.parse() function to the "date" column and format the dates
    df['date'] = df['date'].apply(lambda x: dateparser.parse(x).strftime('%d-%m-%Y'))

    # Calculate Name sentiment and Description sentiment
    df['Name sentiment'] = df['Name'].apply(lambda x: sia.polarity_scores(x)['compound'])
    df['Description sentiment'] = df['description'].apply(lambda x: sia.polarity_scores(x)['compound'])
    
    # Calculate Name subjectivity and Description subjectivity
    df['Name subjectivity'] = df['Name'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    df['Description subjectivity'] = df['description'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    
    # Drop 'Unnamed: 0' column
    df = df.drop('Unnamed: 0', axis=1)
    
    # Rename columns
    df = df.rename(columns={'views': 'Views', 
                            'duration': 'Duration', 
                            'date': 'Date', 
                            'description': 'Description', 
                            'link': 'Link',
                            'Name': 'Title',
                            'Name subjectivity':'Title subjectivity',
                            'Name sentiment':'Title sentiment'})
    
    # Reorder columns
    df = df.reindex(columns=['Title', 
                             'Title sentiment', 
                             'Title subjectivity', 
                             'Views', 
                             'Duration', 
                             'Date', 
                             'Description', 
                             'Description sentiment', 
                             'Description subjectivity', 
                             'Link'])
    
    return df

def cleaning_youtube_comments(df):
    # Initialize SentimentIntensityAnalyzer
    
    try: 
        df.drop("Unnamed: 0", axis=1, inplace=True)
    except:
        pass
    sia = SentimentIntensityAnalyzer()
    df.drop_duplicates(inplace=True)
    df['Date'] = df['Date'].apply(lambda x: BeautifulSoup(x, 'html.parser').text)

    df = df[df['Date'] != 'Unknown'].reset_index(drop=True)
    df['Comment'] = df['Comment'].astype(str)

    if df['Date'].notnull().all():
        df['Date'] = df['Date'].astype(str)

    # apply the dateparser.parse() function to the "date" column and format the dates
    df['Date'] = df['Date'].apply(lambda x: dateparser.parse(x).strftime('%d-%m-%Y') if dateparser.parse(x) is not None else None)

    # Calculate Name sentiment and Description sentiment
    df['Comment sentiment'] = df['Comment'].apply(lambda x: sia.polarity_scores(x)['compound'])

    # Calculate Name subjectivity and Description subjectivity
    df['Comment subjectivity'] = df['Comment'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
     
    # Rename columns
    df = df.rename(columns={'link': 'Link'})

    # Reorder columns
    df = df.reindex(columns=['Username', 
                             'Comment', 
                             'Comment sentiment', 
                             'Comment subjectivity', 
                             'Likes', 
                             'Replies', 
                             'Link'])
    
    return df



In [92]:
df_yt = cleaning_youtube_videos(df_yt)

In [94]:
df_yt.to_csv('./cleaned/df_yt_videos', index=False)

In [12]:
df_comments

,Username,Comment,Likes,Replies,Date,link
1869,chris rutkowski,Insane to think that AI would become a good tu...,0,0,"<a class=""yt-simple-endpoint style-scope yt-fo...",https://www.youtube.com/watch?v=WY518YRfs5M
2946,Idanthyrsus,I like asking it uncomfortable questions. You ...,1,0,"<a class=""yt-simple-endpoint style-scope yt-fo...",https://www.youtube.com/watch?v=_iMItrc0ChU
3085,Dirk Hartmann,Yes AI will change the world\n\nBut all the hy...,0,0,"<a class=""yt-simple-endpoint style-scope yt-fo...",https://www.youtube.com/watch?v=_iMItrc0ChU
1445,soléé,Ohh great a bisexual robot who just wanna be u...,0,0,"<a class=""yt-simple-endpoint style-scope yt-fo...",https://www.youtube.com/watch?v=b2bdGEqPmCI
3053,Goran Josic,"We can say that we live in ""AI bubble"", right ...",1,0,"<a class=""yt-simple-endpoint style-scope yt-fo...",https://www.youtube.com/watch?v=_iMItrc0ChU


In [18]:
df_yt_comments = cleaning_youtube_comments(df_comments)

KeyError: "['Unnamed: 0'] not found in axis"

In [96]:
df_yt_comments.to_csv('./cleaned/df_yt_comments', index=False)

In [34]:
df_yt_comments = cleaning_youtube_comments(df_comments)

In [35]:
df_yt_comments

,Username,Comment,Comment sentiment,Comment subjectivity,Likes,Replies,Link
0,@michaelporter-lo9qc,i have a very hard life . im sick and im lost ...,-0.6232,0.765327,383,100,https://www.youtube.com/watch?v=oz2n-rQCWzI
1,@walterf6993,I truly believe people are being saved by your...,0.9661,0.609762,76,4,https://www.youtube.com/watch?v=oz2n-rQCWzI
2,@gretchen9716,I love your discernment of these times! Your t...,0.9501,0.550909,31,1,https://www.youtube.com/watch?v=oz2n-rQCWzI
3,@melissalarson2569,A couple years ago I couldn't see how the end ...,0.9628,0.950000,44,15,https://www.youtube.com/watch?v=oz2n-rQCWzI
4,@joenorthpeigan3197,Praise the Lord. And Thank you pastor John. T...,0.9719,0.566667,19,0,https://www.youtube.com/watch?v=oz2n-rQCWzI
...,...,...,...,...,...,...,...
38277,Ever4U,I I'll go to Mall to watch items,0.0000,0.000000,0,0,https://www.youtube.com/watch?v=6490tKrGEic
38278,Muneer Hasan,Do lpu really have such kind of teachers...?,0.0000,0.533333,0,1,https://www.youtube.com/watch?v=6490tKrGEic
38279,Soumyaranjan Mendili,Hii sri,0.0000,0.000000,0,0,https://www.youtube.com/watch?v=6490tKrGEic
38280,Yuta Itadori,Bakbakbak,0.0000,0.000000,0,0,https://www.youtube.com/watch?v=6490tKrGEic


In [37]:
df_yt_comments.to_csv('./cleaned/df_yt_comments', index=False)